### Install/import files as needed

In [1]:
!pip install folium

### Import libraries needed

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

### Import pandas data set from previous book

In [3]:
Wiki_postal_df = pd.read_csv(r'C:\Users\MikeNugent78\Desktop\TorontoPostallatlong.csv',index_col=0)

##Wiki_postal_df                ## DEBUGGING CODE - uncomment to see file

### Get the Downtown Toronto Boroughs

In [4]:
Wiki_Toronto_df = Wiki_postal_df.loc[Wiki_postal_df["Borough"] == "Downtown Toronto"]

##Wiki_Toronto_df               ##DEBUGGING CODE - uncomment to see new file

### Get coordinates of Toronto and save

In [5]:
Toronto_address = 'Toronto, CA'

Toronto_geolocator = Nominatim(user_agent="CA_explorer")
Toronto_location = Toronto_geolocator.geocode(Toronto_address)

##Toronto_location = geolocator.geocode(Toronto_address)
Toronto_latitude = Toronto_location.latitude
Toronto_longitude = Toronto_location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(Toronto_latitude, Toronto_longitude))



CLIENT_ID = '5SAQGQTVU3POAZ0XVUMYGSN1WILSSLKE3CMZVZUIEVQUHDBK' # your Foursquare ID
CLIENT_SECRET = 'Z3ACVKGB0Z5UF1E1BPOEJRQOIWYBPNEW4ZBJQ0OMIDFH4DZP' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.
Your credentails:
CLIENT_ID: 5SAQGQTVU3POAZ0XVUMYGSN1WILSSLKE3CMZVZUIEVQUHDBK
CLIENT_SECRET:Z3ACVKGB0Z5UF1E1BPOEJRQOIWYBPNEW4ZBJQ0OMIDFH4DZP


### Retrieve information about local venues

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
       
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

    
    return(nearby_venues)

### Get information for each neighborhood

In [7]:
LIMIT = 100
Toronto_venues = []
Toronto_venues = getNearbyVenues(names=Wiki_Toronto_df['Neighborhood'],
                                   latitudes=Wiki_Toronto_df['Latitude'],
                                   longitudes=Wiki_Toronto_df['Longitude'])


### Remove Venue Category data which are set to "Neighborhood"

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


### There are occurrences of Venue Category with Neighborhood which looks like a data error.
### Find them and remove them

In [8]:
###print (Toronto_venues[Toronto_venues['Venue Category'] == 'Neighborhood'])   ##Debugging Code

## Save the shape to compare later
Old_shape = Toronto_venues.shape

## Remove the 3 redundant Venue Category = Neighborhood entries
Toronto_venues = Toronto_venues[Toronto_venues['Venue Category'] != "Neighborhood"]

## Save the shape to compare later
New_shape=Toronto_venues.shape

##Compare Old and New shapes to make sure records were removed
print("**********************************************")
print("**                                          **")
print("** Rows decreased by the 3 records deleted  **")
print("** Old Shape", Old_shape,'                     **')
print("** New Shape", New_shape,"                     **")
print("**                                          **")
print("**********************************************")

**********************************************
**                                          **
** Rows decreased by the 3 records deleted  **
** Old Shape (1166, 7)                      **
** New Shape (1163, 7)                      **
**                                          **
**********************************************


### Create an empty table of the venue names found

In [9]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
Toronto_onehot.head()

### Add Neighboorhood column
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])

Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head(3)

,Neighborhood,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Create a table of the mean occurrences of the venues

In [10]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.010989,0.021978,0.000000,0.000000,0.000000,0.000000,0.010989,0.000000,0.010989,0.043956,0.000000,0.000000,0.010989,0.032967,0.0,0.0,0.0,0.000000,0.0,0.021978,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.054945,0.0,0.000000,0.000000,0.021978,0.0,0.0,0.010989,0.032967,0.098901,0.0,0.000000,0.0,0.0,0.0,0.010989,0.000000,0.010989,0.0,0.010989,0.000000,0.021978,0.010989,0.000000,0.0,0.010989,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.010989,0.000000,0.0,0.0,0.010989,0.000000,0.000000,0.010989,0.000000,0.000000,0.0,0.010989,0.010989,0.010989,0.0,0.000000,0.0,0.0,0.010989,0.000000,0.000000,0.0,0.0,0.0,0.010989,0.010989,0.010989,0.010989,0.000000,0.0,0.000000,0.000000,0.0,0.054945,0.010989,0.0,0.000000,0.000000,0.010989,0.010989,0.032967,0.010989,0.0,0.000000,0.0,0.0,0.0,0.0,0.010989,0.010989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010989,0.0,0.0,0.0,0.010989,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.010989,0.000000,0.010989,0.000000,0.000000,0.000000,0.0,0.0,0.021978,0.000000,0.0,0.000000,0.032967,0.0,0.000000,0.000000,0.000000,0.010989,0.000000,0.032967,0.000000,0.010989,0.000000,0.0,0.010989,0.000000,0.000000,0.010989,0.010989,0.010989,0.000000,0.000000,0.010989,0.0,0.010989,0.0,0.000000,0.010989,0.010989,0.000000,0.000000,0.0,0.0,0.010989,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.01098

### Create a function to identify the most common venues

In [11]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Populate a table with the top 10 venues for each neighborhood

In [12]:
num_top_venues = 10

# Set up indicators for 1st, 2nd, 3rd, and 4th - 10th
indicators = ['st', 'nd', 'rd','th','th','th','th','th','th','th']

# create columns according to number of top venues
Top_venue_columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
        Top_venue_columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))

# create a new dataframe
TR_hoods_venues_sorted = pd.DataFrame(columns=Top_venue_columns)
TR_hoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    TR_hoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

TR_hoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Japanese Restaurant,Restaurant,Pub
1,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Bar,Italian Restaurant,Café,Gym / Fitness Center,Bakery,Restaurant,Speakeasy,French Restaurant,Bank
2,Central Bay Street,Coffee Shop,Sandwich Place,Middle Eastern Restaurant,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Café,Park,Donut Shop,Breakfast Spot
3,Christie,Grocery Store,Café,Playground,Park,Athletics & Sports,Candy Store,Baby Store,Coffee Shop,Donut Shop,Farmers Market
4,Church and Wellesley,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Diner,Grocery Store,Men's Store,Bubble Tea Shop


### Set up the clusters. 
#### Several runs determined that 6 clusters was the most beneficial when reviewing the neighborhoods

In [13]:
# set number of clusters
kclusters = 7

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 3, 1, 0, 3, 3, 3, 2, 0], dtype=int32)

### Add the clusters back to the venue data
### Then add the location data back to the file for mapping

In [14]:
# add clustering labels

TR_hoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

## Merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood

Toronto_merged = Wiki_Toronto_df
Toronto_merged = Toronto_merged.join(TR_hoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,5,Coffee Shop,Restaurant,Breakfast Spot,Yoga Studio,Thai Restaurant,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,0,Sushi Restaurant,Hobby Shop,Bubble Tea Shop,Burrito Place,Café,Sculpture Garden,Coffee Shop,Ramen Restaurant,College Theater,Park
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,3,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Tea Room,Ramen Restaurant,Pizza Place
15,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,3,Coffee Shop,Café,Seafood Restaurant,Gastropub,American Restaurant,Restaurant,Italian Restaurant,Cocktail Bar,Gym,Clothing Store
20,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754,3,Coffee Shop,Café,Hotel,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Japanese Restaurant,Restaurant,Pub


### Map the clusters

In [15]:
# create map
map_clusters = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))

rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color='#000000',
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
     
map_clusters

### FINDINGS:
### Not surprisingly, the clusters tended towards the economics of a particular area.  
### For instance, the downtown proper is probably supported by office space and has a consistent set of venues appealing to that constituency.  
### Moving just outside of downtown, we probably are looking at living space, rather than office space, and the types of stores there will be simliar to each other but different from the downtown. 
### The specialized areas - the university, the docks, and the hybrid dock location are more specific in their appeal and their venues are notable different from downtown and outer downtown and from each other.

### Cluster 0 (red) - Encircling neighborhoods of downtown proper

In [16]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Queen's Park, Ontario Provincial Government",Sushi Restaurant,Hobby Shop,Bubble Tea Shop,Burrito Place,Café,Sculpture Garden,Coffee Shop,Ramen Restaurant,College Theater,Park
84,"Kensington Market, Chinatown, Grange Park",Café,Vegetarian / Vegan Restaurant,Mexican Restaurant,Bakery,Grocery Store,Art Gallery,Gaming Cafe,Vietnamese Restaurant,Coffee Shop,Pizza Place
87,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Bar,Italian Restaurant,Café,Gym / Fitness Center,Bakery,Restaurant,Speakeasy,French Restaurant,Bank
96,"St. James Town, Cabbagetown",Café,Coffee Shop,Park,Italian Restaurant,Restaurant,Bakery,Pizza Place,General Entertainment,Sandwich Place,Caribbean Restaurant
99,Church and Wellesley,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Diner,Grocery Store,Men's Store,Bubble Tea Shop


### Cluster 1 (purple) - Outlying Northwest neighborhood

In [17]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Christie,Grocery Store,Café,Playground,Park,Athletics & Sports,Candy Store,Baby Store,Coffee Shop,Donut Shop,Farmers Market


### Cluster 2 (royal blue) - Uniquely located on the harbor

In [18]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,"Harbourfront East, Union Station, Toronto Islands",Harbor / Marina,Park,Café,Music Venue,Doner Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


### Cluster 3 (red) - Toronto Downtown proper

In [19]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,"Garden District, Ryerson",Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Tea Room,Ramen Restaurant,Pizza Place
15,St. James Town,Coffee Shop,Café,Seafood Restaurant,Gastropub,American Restaurant,Restaurant,Italian Restaurant,Cocktail Bar,Gym,Clothing Store
20,Berczy Park,Coffee Shop,Café,Hotel,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Japanese Restaurant,Restaurant,Pub
24,Central Bay Street,Coffee Shop,Sandwich Place,Middle Eastern Restaurant,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Café,Park,Donut Shop,Breakfast Spot
30,"Richmond, Adelaide, King",Coffee Shop,Café,Restaurant,Gym,Hotel,Steakhouse,Asian Restaurant,Salad Place,American Restaurant,Japanese Restaurant
42,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Hotel,Café,American Restaurant,Japanese Restaurant,Italian Restaurant,Restaurant,Salad Place,Seafood Restaurant,Breakfast Spot
48,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,Gym,Japanese Restaurant,Seafood Restaurant,Steakhouse,Salad Place,American Restaurant
92,Stn A PO Boxes,Coffee Shop,Restaurant,Gym,Café,Hotel,Deli / Bodega,Japanese Restaurant,Italian Restaurant,Bakery,Breakfast Spot
97,"First Canadian Place, Underground city",Coffee Shop,Café,Restaurant,Hotel,Gym,Japanese Restaurant,Seafood Restaurant,Steakhouse,Salad Place,American Restaurant


### CLuster 4 (light green) - Far North neighborhood

In [20]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Rosedale,Playground,Grocery Store,Park,Candy Store,Dog Run,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


### Cluster 5 (yellow) - East neighborhood but also near the harbor

In [21]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Regent Park, Harbourfront",Coffee Shop,Restaurant,Breakfast Spot,Yoga Studio,Thai Restaurant,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center


### Cluster 6 (orange) - University neighborhood

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 6, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]